 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/2017-09-ece-606-algorithms/ECE%20606%20Quiz%208.ipynb#Quiz-8-Problem-1" data-toc-modified-id="Quiz-8-Problem-1-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Quiz 8 Problem 1</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/2017-09-ece-606-algorithms/ECE%20606%20Quiz%208.ipynb#Simple-Graph-with-2-Shortest-Paths" data-toc-modified-id="Simple-Graph-with-2-Shortest-Paths-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Simple Graph with 2 Shortest Paths</a></span></li><li><span><a href="http://localhost:8888/notebooks/2017-09-ece-606-algorithms/ECE%20606%20Quiz%208.ipynb#Dijkstra's-Algorithm-from-Lecture-9" data-toc-modified-id="Dijkstra's-Algorithm-from-Lecture-9-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Dijkstra's Algorithm from Lecture 9</a></span></li><li><span><a href="http://localhost:8888/notebooks/2017-09-ece-606-algorithms/ECE%20606%20Quiz%208.ipynb#$S$-Does-not-hold-the-Shortest-Path,-it-Holds-ALL-Vertexes-in-$V$" data-toc-modified-id="$S$-Does-not-hold-the-Shortest-Path,-it-Holds-ALL-Vertexes-in-$V$-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>$S$ Does not hold the Shortest Path, it Holds ALL Vertexes in $V$</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/2017-09-ece-606-algorithms/ECE%20606%20Quiz%208.ipynb#Run-with-Debugging-Statements-on" data-toc-modified-id="Run-with-Debugging-Statements-on-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Run with Debugging Statements on</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/2017-09-ece-606-algorithms/ECE%20606%20Quiz%208.ipynb#Converting-$u.\pi$-into-shortest-path" data-toc-modified-id="Converting-$u.\pi$-into-shortest-path-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Converting $u.\pi$ into shortest path</a></span></li><li><span><a href="http://localhost:8888/notebooks/2017-09-ece-606-algorithms/ECE%20606%20Quiz%208.ipynb#Mahesh's-Answer" data-toc-modified-id="Mahesh's-Answer-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Mahesh's Answer</a></span></li></ul></li></ul></div>

# Quiz 8 Problem 1

**Quiz 8, Question 1 (4points):** Two paths $a \overset{p_1}{\leadsto} b$ and $a \overset{p_2}{\leadsto} b$ are considered to be distinct if one of them contains an edge that is not in the other.

Suppose $G = \langle V, E \rangle$ is a connected, unweighted, undirected graph, represented using an adjacency-list. Give an algorithm that takes three inputs: $G, a \in V$, and, $b \in V$, and **outputs two distinct shortest-paths from $a$ to $b$ in $G$**, if they exist, and 'error' otherwise.

Your output can be, for example, as attributes $\pi_1$ and $\pi_2$ for each vertex, i.e., predecessor vertices in corresponding shortest-paths trees. Of course, as $G$ is unweighted, ''shortest-path'' means one of fewest number of edges.

Your algorithm should run in time at worst quadratic in the size of the input. You can assume you have available, as a subroutine, any algorithm we have discussed in the course. Of course, any time your subroutine takes to run has to be part of the running-time of your algorithm.

Note: Remember that both paths must start with $a$ and end with $b$.

## Simple Graph with 2 Shortest Paths

In [114]:
# Example Graph
V1 = {1:{},2:{},3:{},4:{},5:{},6:{}}
E1 = [(1,2), (2,3), (2,4), (3,5), (4,5), (5,6)]
a=1
b=6

* Shortest Path from 1 to 6: [1, 2, 3, 5, 6]
* Alternative Shortest Path from 1 to 6: [1, 2, 4, 5, 6]
<img src="quiz8/graph.png"/>

## Dijkstra's Algorithm from Lecture 9

In [135]:
import copy

infinity = 1000000
nil = -1
def Initialize_Sp(V, E, s):
    for u in V:
        V[u]['d'] = infinity
        V[u]['pi'] = nil
    V[s]['d'] = 0
    
def Extract_Min(V, Q):
    # let u in Q minimize d
    u = nil
    idx = nil
    for i in Q:
        if u == nil:
            u = V[i]
            idx = i
        else:
            if V[i]['d'] < u['d']:
                u = V[i]
                idx = i
    # Q <- Q \ {u}
    Q.remove(idx)
    # return u
    return u, idx
    
def EdgeSearch(E, Q, idx):
    """Returns all v such that: <u,v> in E and v in Q"""
    vs = []
    vs_idx = []
    for e in E:
        if e[0] == idx:
            vs_idx.append(e[1])
        elif e[1] == idx:
            vs_idx.append(e[0])
            
    for v in vs_idx:
        if v in Q:
            vs.append(v)
    
    return vs
    
    
def Relax_SP(V, u, v):
    """updated d and pi for 2 vertexes. u and v are indexes in V"""
    if V[v]['d'] > V[u]['d'] + 1:
        V[v]['d'] = V[u]['d'] + 1
        V[v]['pi'] = u
    
def Dijkstra(V_orig,E_orig,s_orig, debug=True):
    """
    Performs the Dijkstra algotrithm using the graph <V,E> 
    and the starting vertex s.
    
    Will return a modified V, E and from the notes: S (capital s)
    S is returned to show a misunderstanding made in the class
    """
    V = copy.deepcopy(V_orig)
    E = copy.deepcopy(E_orig)
    s = copy.deepcopy(s_orig)
    
    Initialize_Sp(V, E, s)
    S = []
    Q = copy.deepcopy(V.keys())
    
    while len(Q) != 0: # Q != empty set
        # u <- Extract-Min(Q)
        u,idx = Extract_Min(V, Q)
        if debug:
            print 'Extract_Min:', idx, u, '; Q:', Q
        
        # S <- S U {u}
        S.append(idx)
        
        # foreach <u,v> in E, v in Q do
        for v in EdgeSearch(E, Q, idx):
            if debug:
                print '  EdgeSearch: <{},{}>'.format(idx, v)
            # Relax-SP(u,v, 1)
            # assumes weight 1 for all edges
            Relax_SP(V, idx, v)
        if debug:
            for v in V:
                print '  {}:{}'.format(v, V[v])
            print 
    
    return V, E, S

## $S$ Does not hold the Shortest Path, it Holds ALL Vertexes in $V$

At the end of running dijkstra's algorithm the value of $S$ is equal to $V$, **not** the shortest path!

In [136]:
print 'V1:'
for v in V1:
    print '  {}:{}'.format(v, V1[v])

V, E, S = Dijkstra(V1, E1, a, debug=False)
print
print '-'*50
print 
print 'Length of V: {}\nLength of S: {}'.format(len(V), len(S))
print 'S:', S
print 'V:'
for v in V:
    print '  {}:{}'.format(v, V[v])

V1:
  1:{}
  2:{}
  3:{}
  4:{}
  5:{}
  6:{}

--------------------------------------------------

Length of V: 6
Length of S: 6
S: [1, 2, 3, 4, 5, 6]
V:
  1:{'pi': -1, 'd': 0}
  2:{'pi': 1, 'd': 1}
  3:{'pi': 2, 'd': 2}
  4:{'pi': 2, 'd': 2}
  5:{'pi': 3, 'd': 3}
  6:{'pi': 5, 'd': 4}


### Run with Debugging Statements on

In [137]:
V, E, S = Dijkstra(V1, E1, a, debug=True)
print
print '-'*50
print 
print 'Length of V: {}\nLength of S: {}'.format(len(V), len(S))
print 'S:', S
print 'V:'
for v in V:
    print '  {}:{}'.format(v, V[v])

Extract_Min: 1 {'pi': -1, 'd': 0} ; Q: [2, 3, 4, 5, 6]
  EdgeSearch: <1,2>
  1:{'pi': -1, 'd': 0}
  2:{'pi': 1, 'd': 1}
  3:{'pi': -1, 'd': 1000000}
  4:{'pi': -1, 'd': 1000000}
  5:{'pi': -1, 'd': 1000000}
  6:{'pi': -1, 'd': 1000000}

Extract_Min: 2 {'pi': 1, 'd': 1} ; Q: [3, 4, 5, 6]
  EdgeSearch: <2,3>
  EdgeSearch: <2,4>
  1:{'pi': -1, 'd': 0}
  2:{'pi': 1, 'd': 1}
  3:{'pi': 2, 'd': 2}
  4:{'pi': 2, 'd': 2}
  5:{'pi': -1, 'd': 1000000}
  6:{'pi': -1, 'd': 1000000}

Extract_Min: 3 {'pi': 2, 'd': 2} ; Q: [4, 5, 6]
  EdgeSearch: <3,5>
  1:{'pi': -1, 'd': 0}
  2:{'pi': 1, 'd': 1}
  3:{'pi': 2, 'd': 2}
  4:{'pi': 2, 'd': 2}
  5:{'pi': 3, 'd': 3}
  6:{'pi': -1, 'd': 1000000}

Extract_Min: 4 {'pi': 2, 'd': 2} ; Q: [5, 6]
  EdgeSearch: <4,5>
  1:{'pi': -1, 'd': 0}
  2:{'pi': 1, 'd': 1}
  3:{'pi': 2, 'd': 2}
  4:{'pi': 2, 'd': 2}
  5:{'pi': 3, 'd': 3}
  6:{'pi': -1, 'd': 1000000}

Extract_Min: 5 {'pi': 3, 'd': 3} ; Q: [6]
  EdgeSearch: <5,6>
  1:{'pi': -1, 'd': 0}
  2:{'pi': 1, 'd': 1}
  

## Converting $u.\pi$ into shortest path

After running Dijkstra's algorithm, it is easily to convert the new graph (new because there are new descriptors for each vertex) into a shortest path

In [138]:
def shortestPath(V, a, b, pi='pi'):
    """
    Returns a list of vertexes which is the shortest path from a to b for graph V.
    NOTE: Dijkstra's algorithm must be run on V
    """
    if V[b][pi] == nil or V[b]['d'] == infinity:
        return 'error: no path'
    
    path = [b]
    while True:
        t = path[0]
        if t == a:
            return path
        if t == nil:
            return '[ERROR] Reached the root, no parents to follow. Unable to create a path from {} to {}'.format(a,b)
            
        path.insert(0, V[t][pi])

In [139]:
print 'Shortest path from {} to {}: {}'.format(a,b,shortestPath(V, a, b))
print 'Shortest path from {} to {}: {}'.format(a,4,shortestPath(V, a, 4))

Shortest path from 1 to 6: [1, 2, 3, 5, 6]
Shortest path from 1 to 4: [1, 2, 4]


Be careful that the shortest path algorithm relies on Dijkstra's algorithm being run with the same $a$ input. 

Above going from 1 to 4 is still correct, but below is an example where it is wrong:

In [140]:
print 'Shortest path from {} to {}: {}'.format(3,4,shortestPath(V, 3,4))

Shortest path from 3 to 4: [ERROR] Reached the root, no parents to follow. Unable to create a path from 3 to 4


We can fix this issue by running dijkstra's algorithm with the correct parameters:

In [141]:
V3, _, _ = Dijkstra(V1, E1, 3, debug=False)
print 'Shortest path from {} to {}: {}'.format(3,4,shortestPath(V3, 3,4))

Shortest path from 3 to 4: [3, 2, 4]


## Mahesh's Answer

AnotherSP$(G, a, b)$
1. *Bfs*$(G, a, \pi_1)$
1. $u \gets b$
1. **While**($u \not= a$){
1. &nbsp;&nbsp;&nbsp;&nbsp; $E^\prime \gets E \setminus \{\langle u.\pi_1, u \rangle\}$
1. &nbsp;&nbsp;&nbsp;&nbsp; *Bfs*$(\langle V, E^\prime \rangle, a, \pi_2)$
1. &nbsp;&nbsp;&nbsp;&nbsp; **If**( $b.\pi_2 \not=$ **Nil** ) { **Return** the $\pi_1, \pi_2$ values }
1. &nbsp;&nbsp;&nbsp;&nbsp; $u \gets u.\pi_1$
1. }
1. **Return** error

In [142]:
pi_1 = 'pi_1'
pi_2 = 'pi_2'
W = 'white'
G = 'gray'
B = 'black'
NIL = -1

def FindEdges(E, u):
    """Given u and E: find all v such that <u,v> in E"""
    vs = []
    for e in E:
        if e[0] == u:
            vs.append(e[1])
        elif e[1] == u:
            vs.append(e[0])
    return vs

def bfs(V, E, s, pi=pi_1):
    """
    Performs a breath first search (BFS) on graph <V,E>, 
    starting at vertex s and applying the parents to variable pi
    """
    for u in V:
        V[u]['colour'] = W
        V[u]['d'] = infinity
        V[u][pi] = NIL
    V[s]['colour'] = G
    V[s]['d'] = 0
    V[s][pi] = NIL
    
    Q = [s]
    while len(Q) != 0:
        u = Q.pop()
        
        for v in FindEdges(E, u):
            if V[v]['colour'] == W:
                V[v]['colour'] = G
                V[v]['d'] = V[u]['d'] + 1
                V[v][pi] = u
                Q.append(v)
        
        V[u]['colour'] = B

def CopyAndRemoveEdge(E, v, u):
    E_ = copy.deepcopy(E)
    
    for e in E_:
        if e[0] == v and e[1] == u:
            # if <v,u> in E_
            E_.remove(e)
        elif e[1] == v and e[0] == u:
            # if <u,v> in E_
            E_.remove(e)
    
    return E_
    
def AnotherSP(V, E, a, b):
    bfs(V,E, a, pi=pi_1)
    u = b
    
    while(u != a):
        E_ = CopyAndRemoveEdge(E, V[u][pi_1], u) # E' <- E \ {u.pi, u}
        bfs(V, E_, a, pi=pi_2)
        
        if(V[b][pi_2] != NIL):
            return 'success'
        
        u = V[u][pi_1]
    
    return 'error'
    

In [145]:
V_test = copy.deepcopy(V1)
E_test = copy.deepcopy(E1)

print 'V1:'
for v in V1:
    print '  {}:{}'.format(v, V1[v])

print 'AnotherSP:', AnotherSP(V_test, E_test, a, b)

print 'V:'
for v in V_test:
    print '  {}:{}'.format(v, V_test[v])
    
print 'Shortest path from {} to {} (using {}): {}'.format(a,b,pi_1, shortestPath(V_test, a, b, pi=pi_1))
print 'Shortest path from {} to {} (using {}): {}'.format(a,b,pi_2, shortestPath(V_test, a, b, pi=pi_2))

V1:
  1:{}
  2:{}
  3:{}
  4:{}
  5:{}
  6:{}
AnotherSP: success
V:
  1:{'pi_1': -1, 'colour': 'black', 'pi_2': -1, 'd': 0}
  2:{'pi_1': 1, 'colour': 'black', 'pi_2': 1, 'd': 1}
  3:{'pi_1': 2, 'colour': 'black', 'pi_2': 2, 'd': 2}
  4:{'pi_1': 2, 'colour': 'black', 'pi_2': 2, 'd': 2}
  5:{'pi_1': 4, 'colour': 'black', 'pi_2': 3, 'd': 3}
  6:{'pi_1': 5, 'colour': 'black', 'pi_2': 5, 'd': 4}
Shortest path from 1 to 6 (using pi_1): [1, 2, 4, 5, 6]
Shortest path from 1 to 6 (using pi_2): [1, 2, 3, 5, 6]
